## ToDo

* plot eigenvalue by PC component to pick how many

## packages

In [1]:
# SQL
import sqlite3

# general
import numpy as np
import pandas as pd 
import datetime

# packages for text analysis
import gensim
import nltk
import re
import string
#nltk.download('punkt')
#nltk.download('stopwords')

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# LSA via SVD
from sklearn.decomposition import TruncatedSVD

---
## load data
from http://www.trumptwitterarchive.com/archive

In [2]:
def load_data():
    d = pd.read_csv('./data/tweets_all.csv')
    d = d[['text', 'created_at']]
    d.columns = ['text', 'date']
    return d

,text,date
0,RT @GOPChairwoman: Democrats in Congress have ...,02-26-2020 16:35:08
1,RT @GOPChairwoman: People are tired of Democra...,02-26-2020 16:34:26
2,My two great friends. Proud of you both! https...,02-26-2020 16:20:43
3,Eric I can live with that! https://t.co/TtNdK9...,02-26-2020 16:17:52
4,Piers I like you too! https://t.co/pUe09YlrY0,02-26-2020 16:15:57
5,RT @tonykatz: Check out the press conference o...,02-26-2020 16:07:42
6,Big Rally in the Great State of South Carolina...,02-26-2020 13:51:13
7,“Every poll you look at shows that Black suppo...,02-26-2020 13:41:24
8,Big Story Big Win - Except in the Fake News wh...,02-26-2020 13:35:32
9,....during a debate). Pocahontas was mean &amp...,02-26-2020 13:24:23


---
## cleaning

In [3]:
def clean_data(d):
    c = d.copy()

    ## cleaning from issues in raw data

    # missing dates
    c = c.loc[c['date'].notnull(), :]

    # unreadable dates
    c['date'] = pd.to_datetime(c['date'], errors='coerce')
    c = c.loc[c['date'].notnull(), :]

    # removing ampersand text
    c['text'] = c['text'].str.replace('&amp;', '')
    
    return c

# split tweets and retweets
def split_retweets(d):
    cond = d['text'].str.find('RT', 0, 2) != -1 #retweets
    twts = c.loc[~cond, :].reset_index(drop=True)
    rtwts = c.loc[cond, :].reset_index(drop=True)
    return twts, rtwts

,text,date
0,RT @GOPChairwoman: Democrats in Congress have ...,2020-02-26 16:35:08
1,RT @GOPChairwoman: People are tired of Democra...,2020-02-26 16:34:26


## create term vectors, remove stop words, and stem

In [289]:
def twts_to_tvs(twts):
    '''turns tweets into term vectors'''
    tvs = twts.apply(twt_clean_split_to_tv)
    tvs = tvs.apply(tv_remove_stopwords)
    tvs = tvs.apply(tv_stem)
    return tvs

def twt_clean_split_to_tv(twt):
    '''cleans characters and splits into term vector'''
    twt = twt.lower() # lower case
    twt = re.sub(r'http\S+', '', twt) # remove URL
    twt = re.sub('\d+', '', twt) # remove digits
    twt = re.sub(r'\B#\w*[a-zA-Z]+\w*', '', twt) # remove hashtag
    twt = re.sub('@[^\s]+','', twt) # remove @username

    # odd characters found not in string.punctuation
    odd_chars = ('“', '”', '’', '‘')
    chrs = string.punctuation.join(odd_chars)
    twt = (re.compile('[%s]' % re.escape(chrs))
             .sub('', twt))
    
    twt = nltk.word_tokenize(twt)
    return twt

def create_swords():
    '''function for defining stop words to be used'''
    
    # these are largely chosen when they were found to 
    # obscure the meaning of a topic grouping in latent
    # semantic analysis
    r_names = ['donald', 'trump', 'fox']
    r_politics = ['democrat', 'democrats', 
                  'republican', 'republicans',
                  'maga', 'president', 'presidents', 'presidency',
                  'us', 'state', 'states', 'country', 'countries',
                  'vote', 'usa']
    r_nonwords = ['pm', 'pme']
    r_numwords = ['one', 'two', 'three']
    r_days = ['monday', 'tuesday', 'wednesday', 'thursday',
              'friday', 'saturday', 'sunday',
              'morning', 'night', 'tonight', 
              'day', 'week', 'year',
              'today']
    r_other = ['make', 'america', 'great', 'again',
               'thank', 'thanks', 'you', 'tonight', 'get', 'go',
               'people', 'new', 'news', 'twitter', 'media',
               'much', 'good', 'big', 'want', 'look', 'like',
               'many', 'morning', 'tonight', 'night', 'time',
               'never', 'would', 'back', 'go', 'even',
               'one', 'going']
    
    rmv = (r_names + r_politics + r_nonwords + 
           r_numwords + r_days + r_other)
    swords = nltk.corpus.stopwords.words('english') + rmv
    
    swords = [re.sub('[^A-Za-z0-9]+', '', s) 
              for s in swords] # remove punc
    
    return swords

def tv_remove_stopwords(tv):
    swords = create_swords()
    newtv = []
    for t in tv:
        if t not in swords:
            newtv.append(t)
    return newtv

def tv_stem(tv):
    '''stem a term vector'''
    
    stemmer = nltk.stem.porter.PorterStemmer()
    for i in range(0, len(tv)):
        tv[i] = stemmer.stem(tv[i])
    return tv

def twtsdf_to_tvsdf(twtsdf, min_vec_len):
    '''adds column of term vectors to df with tweets'''
    twtsdf = twtsdf.copy()
    twtsdf['tvs'] = twts_to_tvs(twtsdf['text'])
    twtsdf = twtsdf[twtsdf['tvs'].map(len) >= min_vec_len]
    twtsdf = twtsdf.reset_index(drop=True)
    return twtsdf

## similarity of terms

calculate TF-IDF then perform SVD

In [281]:
def tvs_to_svd(tvs, num_features, num_comps):
    '''take term vectors (tvs) and perform svd on tf-idf'''
    tvs = list(tvs.apply(lambda x: ' '.join(x)))
    
    v = TfidfVectorizer(max_features=num_features) # need to pass out to get feature names
    tfidf = v.fit_transform(tvs)
    
    svd = TruncatedSVD(n_components=num_comps, 
                       algorithm='randomized', 
                       n_iter=100, random_state=123)
    svd.fit(tfidf)    
    return svd, v

def print_top_topics(svd, v, num_comps, num_words):
    
    terms = v.get_feature_names()

    for i, comp in enumerate(svd.components_[:num_comps]):
        terms_comp = zip(terms, comp)
        sorted_terms = sorted(terms_comp, key=lambda x: x[1], reverse=True)[:num_words]
        print("\nTopic "+str(i)+": ")
        for t in sorted_terms:
            print(t[0], end=' ')
        print()

---
## Story

### Topics by Words from Latent Semantic Analysis

0. load data

1. create term vectors

2. calculate TF-IDF and perform SVD on it

### data
Trump tweets from http://www.trumptwitterarchive.com/archive

In [290]:
d = load_data()
c = clean_data(d)
dt, drt = split_retweets(c) #split twts and re-twts

# campaign annoucement to election day
cond = ((dt['date'] >= pd.Timestamp('2015-07-16'))
        & (dt['date'] <= pd.Timestamp('2016-11-08')))

d = dt[cond]
d.head(3)

,text,date
11120,LIVE on #Periscope: Join me for a few minutes ...,2016-11-07 23:28:48
11121,Hey Missouri let's defeat Crooked Hillary @ko...,2016-11-07 22:21:53
11122,'America must decide between failed policies o...,2016-11-07 21:37:25


### create term vectors

In [291]:
# arg 2 - minmum number of terms to use twt in analysis
tvsdf = twtsdf_to_tvsdf(d, min_vec_len=10)
tvsdf.head(3)

,text,date,tvs
0,Hey Missouri let's defeat Crooked Hillary @ko...,2016-11-07 22:21:53,"[hey, missouri, let, defeat, crook, hillari, k..."
1,Just landed in North Carolina- heading to the ...,2016-11-07 19:30:12,"[land, north, carolina, head, js, dorton, aren..."
2,‘Must Act Immediately’: Clinton Charity Lawyer...,2016-11-05 21:38:01,"[must, act, immedi, clinton, chariti, lawyer, ..."


### calculate TF-IDF and perform SVD

In [292]:
# arg 2 - number of features
# arg 3 - number of PCs
svd, v = tvs_to_svd(tvsdf['tvs'], 
                    num_features=1000, 
                    num_comps=100) 

### results

In [286]:
# arg 3 - number of topics
# arg 4 - number of top words from PC to show
print_top_topics(svd, v, 10, 30)


Topic 0: 
border hillari job fals work fake total win american bad clinton deal must come year crime elect done crook wall china need tax report said senat secur militari say dem 

Topic 1: 
border wall secur militari crime strong tax need mexico immigr must job vet southern endors law cut amend drug senat nation stop unit love come congress open build trade billion 

Topic 2: 
china trade deal tariff korea billion dollar north year meet unit product negoti farmer compani fake world xi continu well american economi made work come better iran pay forward happen 

Topic 3: 
fals android poll donaldtrump watch cruz love show web win interview support lead debat ted gop rubio rate need jeb number bush mr iphonethank best american iowa amaz im crowd 

Topic 4: 
hillari china fals clinton crook trade deal android wall border tariff billion dollar mexico secur unit must immigr product email southern korea berni stop come pay farmer negoti build xi 

Topic 5: 
job tax hillari cut endors crook

---

---

## loads tweets intos SQLite DB

In [ ]:
# make db and set cursor
conn = sqlite3.connect('./data/tweets.db')
cur = conn.cursor()

# delete table if it exists
try:
    cur.execute('''DROP TABLE twts''')
except:
    pass

# make table
cur.execute('''CREATE TABLE twts
            ([text] text, 
             [date] datetime,
             [id_str] integer PRIMARY KEY)''')

print()

In [ ]:
c.to_sql('twts', conn, if_exists='append', index = False)

## queries

In [ ]:
cur.execute('''
SELECT *
FROM twts AS t
WHERE date > DATE('2018-01-01') AND
      text NOT LIKE 'RT%'
LIMIT 5 
''')

dnew = cur.fetchall()

In [ ]:
pd.DataFrame(dnew)